In [1]:
import json

with open('documents-with-ids.json', 'rt') as f_in:
    documents = json.load(f_in)

In [2]:
documents

[{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp',
  'doc_id': 'bae7a31e6abaddb52b4061dcf238fc61'},
 {'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
  'section': 'General course-related questions',
  'question': 'Course - What are the prerequisites for this course?',
  'course': 'data-engineering-zoomcamp',
  'doc_id': '3e5d4959603c68a1e154fa2a6bd9d1e8'},
 {'text': "Yes, even if you don't register, 

In [ ]:
# from elasticsearch import Elasticsearch

# es_client = Elasticsearch('http://localhost:9200') 

# index_settings = {
#     "settings": {
#         "number_of_shards": 1,
#         "number_of_replicas": 0
#     },
#     "mappings": {
#         "properties": {
#             "text": {"type": "text"},
#             "section": {"type": "text"},
#             "question": {"type": "text"},
#             "course": {"type": "keyword"},
#             "id": {"type": "keyword"},
#         }
#     }
# }

# index_name = "course-questions"

# es_client.indices.delete(index=index_name, ignore_unavailable=True)
# es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [ ]:
# from tqdm.auto import tqdm

# for doc in tqdm(documents):
#     es_client.index(index=index_name, document=doc)

In [ ]:
# def elastic_search(query, course):
#     search_query = {
#         "size": 5,
#         "query": {
#             "bool": {
#                 "must": {
#                     "multi_match": {
#                         "query": query,
#                         "fields": ["question^3", "text", "section"],
#                         "type": "best_fields"
#                     }
#                 },
#                 "filter": {
#                     "term": {
#                         "course": course
#                     }
#                 }
#             }
#         }
#     }

#     response = es_client.search(index=index_name, body=search_query)
    
#     result_docs = []
    
#     for hit in response['hits']['hits']:
#         result_docs.append(hit['_source'])
    
#     return result_docs

# elastic_search(
#     query="I just discovered the course. Can I still join?",
#     course="data-engineering-zoomcamp"
# )

In [ ]:
from rag.search import elasticsearch_client as es

es_client = es.ElasticsearchClient()

es_client.index_documents(documents, "course-questions")

Indexing documents:   0%|          | 0/948 [00:00<?, ?it/s]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'doc_id': 'bae7a31e6abaddb52b4061dcf238fc61'}

In [11]:
if es_client.index_exists("course-questions"):
    doc = es_client.get_document("bae7a31e6abaddb52b4061dcf238fc61", "course-questions")
    print(doc)
else:
    print("Index does not exist")

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.", 'section': 'General course-related questions', 'question': 'Course - When will the course start?', 'course': 'data-engineering-zoomcamp', 'doc_id': 'bae7a31e6abaddb52b4061dcf238fc61'}


In [17]:
from rag.pipeline.rag import RAGPipeline
from rag.models.course import Course

question = "I just discovered the course. Can I still join?"

rag = RAGPipeline()

result = rag.search(question=question, course_filter=Course.DATA_ENGINEERING_ZOOMCAMP, boost=3)

result

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp',
  'doc_id': '60a31bbef930b3d6b127405fcd0b618e',
  'full_text': "Course - Can I still join the course after the start date? Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subje

In [18]:
import pandas as pd

In [66]:
df_ground_truth = pd.read_csv('ground-truth-data.csv')

In [67]:
df_ground_truth

,question,course,document
0,When is the exact start date and time for the ...,data-engineering-zoomcamp,bae7a31e6abaddb52b4061dcf238fc61
1,Can you tell me how to subscribe to the course...,data-engineering-zoomcamp,bae7a31e6abaddb52b4061dcf238fc61
2,What steps should I take before the course beg...,data-engineering-zoomcamp,bae7a31e6abaddb52b4061dcf238fc61
3,How can I join the Telegram channel for course...,data-engineering-zoomcamp,bae7a31e6abaddb52b4061dcf238fc61
4,Where should I register to join the course dis...,data-engineering-zoomcamp,bae7a31e6abaddb52b4061dcf238fc61
...,...,...,...
4674,What steps are required to dismantle AWS infra...,mlops-zoomcamp,229c6b390dce6d14c9344e792ca1ec08
4675,Can you describe the process to demolish Terra...,mlops-zoomcamp,229c6b390dce6d14c9344e792ca1ec08
4676,What initial command should be executed to con...,mlops-zoomcamp,229c6b390dce6d14c9344e792ca1ec08
4677,How do you specify variable files while destro...,mlops-zoomcamp,229c6b390dce6d14c9344e792ca1ec08


In [68]:
ground_truth = df_ground_truth.to_dict(orient='records')

In [69]:
ground_truth

[{'question': 'When is the exact start date and time for the course?',
  'course': 'data-engineering-zoomcamp',
  'document': 'bae7a31e6abaddb52b4061dcf238fc61'},
 {'question': 'Can you tell me how to subscribe to the course Google Calendar?',
  'course': 'data-engineering-zoomcamp',
  'document': 'bae7a31e6abaddb52b4061dcf238fc61'},
 {'question': 'What steps should I take before the course begins?',
  'course': 'data-engineering-zoomcamp',
  'document': 'bae7a31e6abaddb52b4061dcf238fc61'},
 {'question': 'How can I join the Telegram channel for course announcements?',
  'course': 'data-engineering-zoomcamp',
  'document': 'bae7a31e6abaddb52b4061dcf238fc61'},
 {'question': 'Where should I register to join the course discussion channel?',
  'course': 'data-engineering-zoomcamp',
  'document': 'bae7a31e6abaddb52b4061dcf238fc61'},
 {'question': 'Where can I find the prerequisites for the course?',
  'course': 'data-engineering-zoomcamp',
  'document': '3e5d4959603c68a1e154fa2a6bd9d1e8'},
 

In [70]:
import importlib
import rag.pipeline.rag
importlib.reload(rag.pipeline.rag)

from tqdm.auto import tqdm
from rag.pipeline.rag import RAGPipeline

rag = RAGPipeline()

relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    course = q['course']
    # results = elastic_search(query=q['question'], course=q['course'])
    results = rag.search(question=q['question'], course_filter=course, boost=3)
    relevance = [d['doc_id'] == doc_id for d in results]
    relevance_total.append(relevance)

  0%|          | 0/4679 [00:00<?, ?it/s]

In [71]:
relevance_total

[[False, True, False, False, False],
 [False, False, False, False, False],
 [False, False, False, False, False],
 [False, False, False, False, False],
 [False, False, False, False, False],
 [False, True, False, False, False],
 [False, True, False, False, False],
 [True, False, False, False, False],
 [True, False, False, False, False],
 [True, False, False, False, False],
 [False, False, False, False, False],
 [True, False, False, False, False],
 [False, False, False, False, False],
 [False, False, False, False, False],
 [False, False, False, False, False],
 [True, False, False, False, False],
 [True, False, False, False, False],
 [True, False, False, False, False],
 [True, False, False, False, False],
 [True, False, False, False, False],
 [True, False, False, False, False],
 [False, False, False, False, False],
 [False, False, True, False, False],
 [False, False, True, False, False],
 [False, False, False, False, False],
 [False, True, False, False, False],
 [False, False, False, False

In [72]:
example = [
    [True, False, False, False, False], # 1, 
    [False, False, False, False, False], # 0
    [False, False, False, False, False], # 0 
    [False, False, False, False, False], # 0
    [False, False, False, False, False], # 0 
    [True, False, False, False, False], # 1
    [True, False, False, False, False], # 1
    [True, False, False, False, False], # 1
    [True, False, False, False, False], # 1
    [True, False, False, False, False], # 1 
    [False, False, True, False, False],  # 1/3
    [False, False, False, False, False], # 0
]

# 1 => 1
# 2 => 1 / 2 = 0.5
# 3 => 1 / 3 = 0.3333
# 4 => 0.25
# 5 => 0.2
# rank => 1 / rank
# none => 0

- hit-rate (recall)
- Mean Reciprocal Rank (mrr)

In [73]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [74]:
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [75]:
hit_rate(relevance_total), mrr(relevance_total)

(0.7285744817268647, 0.5898589442188503)

In [76]:
from rag.search import qdrant_client_custom as qd

qd_client = qd.QdrantClientCustom()

qd_client.collection_exists("zoomcamp-courses-questions")

True

In [77]:
from rag.pipeline.rag import RAGPipeline
from rag.models.course import Course

question = "I just discovered the course. Can I still join?"

rag = RAGPipeline()

result = rag.search_vector(question=question, course_filter=Course.DATA_ENGINEERING_ZOOMCAMP)

result

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'question': 'Course - Can I still join the course after the start date?',
  'section': 'General course-related questions',
  'course': 'data-engineering-zoomcamp',
  'doc_id': '60a31bbef930b3d6b127405fcd0b618e'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'question': 'Course - Can I follow the course after it finishes?',
  'section': 'General course-related questions',
  'course': 'data-engineering-zoomcamp',
  'doc_id': 'cb86516adcdcafa29f0758ae6ca28a0b'},
 {'text': "The purpose of this document is to capture

In [ ]:
import importlib
import rag.pipeline.rag
importlib.reload(rag.pipeline.rag)

from tqdm.auto import tqdm
from rag.pipeline.rag import RAGPipeline

rag = RAGPipeline()

relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    course = q['course']
    results = rag.search_vector(question=q['question'], course_filter=course)
    relevance = [d['doc_id'] == doc_id for d in results]
    relevance_total.append(relevance)

  0%|          | 0/4679 [00:00<?, ?it/s]

When is the exact start date and time for the course?
Can you tell me how to subscribe to the course Google Calendar?
What steps should I take before the course begins?
How can I join the Telegram channel for course announcements?
Where should I register to join the course discussion channel?
Where can I find the prerequisites for the course?
What should I review before starting this course?
Is there a location where I can see the course prerequisites?
What prior knowledge is required for this course?
Where are the details on the course prerequisites listed?
Can I participate without registering?
Is it possible to join after the course begins?
Are there consequences for late registration?
Will I be able to submit assignments without prior registration?
Is there a deadline for project submissions?
When should I expect to get a confirmation email after enrolling in the Data Engineering Bootcamp?
Do I need to wait for a confirmation email to start the Data Engineering Bootcamp?
Is the reg

In [40]:
hit_rate(relevance_total), mrr(relevance_total)

(0.7722066133563864, 0.661454506159499)

Compare with ES results:
```
(0.7395720769397017, 0.6032418413658963)
```

In [42]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [44]:
evaluate(ground_truth, lambda q: elastic_search(q['question'], q['course']))

  0%|          | 0/4627 [00:00<?, ?it/s]

{'hit_rate': 0.7395720769397017, 'mrr': 0.6032418413658963}

In [45]:
evaluate(ground_truth, lambda q: minsearch_search(q['question'], q['course']))

  0%|          | 0/4627 [00:00<?, ?it/s]

{'hit_rate': 0.7722066133563864, 'mrr': 0.661454506159499}